# Projections

- Notebook illustrating how to do projections in python with a few different libraries

In [1]:
import rasterio

In [2]:
dataset = rasterio.open('../data/raw/S2A_17TPM_20210730_0_L2A_WVP.tif')

RasterioIOError: ../data/raw/S2A_17TPM_20210730_0_L2A_WVP.tif: No such file or directory

In [ ]:
dataset.crs

CRS.from_epsg(32617)

In [ ]:
dataset.bounds

BoundingBox(left=600000.0, bottom=5090220.0, right=709800.0, top=5200020.0)

In [ ]:
bounding_box = dataset.bounds

In [ ]:
bounding_box.bottom

5090220.0

One way to reproject the bounding box - create your own function using pyproj

In [ ]:
from pyproj import Transformer

In [ ]:

transformer = Transformer.from_crs("epsg:32617", 
                                   "epsg:4326",
                                   always_xy=True,
                                  ) 



In [ ]:
lon, lat, btm_lon,btm_lat = transformer.transform(bounding_box.bottom, bounding_box.left, bounding_box.bottom, bounding_box.right)


In [ ]:
from rasterio import warp

In [ ]:
bounding_box

BoundingBox(left=600000.0, bottom=5090220.0, right=709800.0, top=5200020.0)

In [ ]:
bounds_trans = warp.transform_bounds(dataset.crs,{'init': 'epsg:4326'},*bounding_box)
print(bounds_trans)

(-79.70951593127886, 45.93349650441821, -78.244415083948, 46.94616825350531)


## Create a Polygon from Bounding Box

In [ ]:
def generate_poly(bbox):
    """
    Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
    """
    return [
        [bbox[1], bbox[0]],
        [bbox[1],bbox[2]],
        [bbox[3],bbox[2]],
        [bbox[3], bbox[0]],
        [bbox[1], bbox[0]]
    ]

In [ ]:
poly = generate_poly(bounds_trans)
poly

[[45.93349650441821, -79.70951593127886],
 [45.93349650441821, -78.244415083948],
 [46.94616825350531, -78.244415083948],
 [46.94616825350531, -79.70951593127886],
 [45.93349650441821, -79.70951593127886]]

In [ ]:
bounds_trans

(-79.70951593127886, 45.93349650441821, -78.244415083948, 46.94616825350531)

In [ ]:
# Create Polygon
pol_bounds_trans = generate_polygon(bounds_trans)
pol_bounds_trans

[[-79.70951593127886, 45.93349650441821],
 [-78.244415083948, 45.93349650441821],
 [-78.244415083948, 46.94616825350531],
 [-79.70951593127886, 46.94616825350531],
 [-79.70951593127886, 45.93349650441821]]

## Visualize in Folium

In [ ]:
import folium

In [ ]:

# original bounds of the image

polyline_polygon_trans_original = folium.PolyLine(poly,
                                                  popup="polygon_trans_original",
                                                  color="orange")


mean_lat = (bounds_trans[1] + bounds_trans[3]) / 2.0
mean_lng = (bounds_trans[0] + bounds_trans[2]) / 2.0
map_bb = folium.Map(location=[mean_lat,mean_lng],
                zoom_start=6)
map_bb.add_child(polyline_polygon_trans_original)
map_bb

A second way:

In [ ]:
import rioxarray 
import xarray

In [ ]:
xds_lonlat = bounding_box.rio.reproject("EPSG:4326")

AttributeError: 'BoundingBox' object has no attribute 'rio'

In [ ]:
dataset.rio.bounds()

AttributeError: 'DatasetReader' object has no attribute 'rio'

In [ ]:
from shapely.geometry import box
geom = box(*bounding_box)
print(geom.wkt)

POLYGON ((709800 5090220, 709800 5200020, 600000 5200020, 600000 5090220, 709800 5090220))


In [ ]:
import pyproj

from shapely import Point
from shapely.ops import transform



ImportError: cannot import name 'Point' from 'shapely' (/home/lauren/miniconda3/envs/pystac-env/lib/python3.10/site-packages/shapely/__init__.py)

In [ ]:
crs = CRS.from_epsg(4326)
crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
from pystac_client import Client
#catalog or api?
catalog = Client.open("https://earth-search.aws.element84.com/v0")

In [ ]:
mysearch = catalog.search(
    collections=['sentinel-s2-l1c'],
    intersects =geom,
    max_items=10)
print(f"{mysearch.matched()} items found")

0 items found
